In [1]:
import pandas as pd
import numpy as np
from ipynb.fs.defs.add_features import add_features
import os
import pandas_datareader as pdr
import time

In [2]:
def stock_scan(api):
    
    """ Scan of all the stocks in the NYSE and NASDAQ to find
    stocks that meet the criteria
    
    Parameters: 
    api(string): Tiingo API key
    
    Returns a generated list of tickers that meet the following:

    Bullish Trend: 20 MA > 50 MA > 200 MA
    Bollinger Band Squeeze: Width of bollinger bands < 6 percent
    High Volume: Volume over 500K a day, leads to less volatility
    Close to All Time High: Within 10 percent of all time high
    Minimum Yearly ROI: ROI of minimum of 10 percent in last year
    Close to 50MA: Within 2 percent above 50MA
    """
    start_time = time.time()
    
    # Import the live NASDAQ stock list from nasdaq.com
    nasdaq_url = 'https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nasdaq&render=download'
    nasdaq_df = pd.read_csv(nasdaq_url)

    print('There are {} stocks in the NASDAQ'.format(len(nasdaq_df)))
    
    # Import the live NYSE stock list from nasdaq.com
    nyse_url = 'https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nyse&render=download'
    nyse_df = pd.read_csv(nyse_url)

    print('There are {} stocks in the NYSE'.format(len(nyse_df)))
    
    # Concat both the NYSE and NASDAQ stocks 
    col = ['Symbol', 'Name', 'MarketCap', 'Sector', 'industry']
    df = pd.concat([nasdaq_df[col], nyse_df[col]])
    
    # Drop preferred stocks
    df = df[~(df['Symbol'].str.contains('^', regex=False)) 
            & ~(df['Symbol'].str.contains('.', regex=False))
            & ~(df['Symbol'].str.contains('~', regex=False))]

    print('There are a combined total of {} stocks'.format(df.shape[0]))
    
    # list of stocks
    stock_list = []
    
    # Generate timeframe OHLCV datasets for each ticker
    for x in df.iloc[:,0]:
        try:
            tmp_df = pdr.get_data_tiingo(x, api_key=api)
            tmp_df = tmp_df.tail(1000)

            # Add additional features
            tmp_df = add_features(tmp_df)

            # Append to stock list if meets the following parameters
            # 20MA > 50MA > 200MA
            if tmp_df['20MA'].iloc[-1] > tmp_df['50MA'].iloc[-1] > tmp_df['200MA'].iloc[-1]:

                # Bollinger band squeeze of 6 percent or less
                if tmp_df['width_%'].iloc[-1] < 6:

                    # Average volume of over 500K
                    if  tmp_df['avg_volume'].iloc[-1] > 500000:

                        # Within 10 percent of all time high
                        if (tmp_df['close'].iloc[-1] / np.max(tmp_df['close'])) > .90:

                            # Stock has ROI of 10 percent in last year
                            if (tmp_df['close'].iloc[-1]/ tmp_df['close'].iloc[-253]) > 1.10:              
                                
                                # Within 2 percent of the 50MA
                                if (1.02*tmp_df['50MA'].iloc[-1]) > tmp_df['close'].iloc[-1] > tmp_df['50MA'].iloc[-1]:
                                    stock_list.append(x)
                                    
        except:
            pass
    
    print("\n--- %s seconds ---" % (time.time() - start_time))  
    return stock_list